In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


df= pd.read_csv(r"D:\DEPAUL\OTHER\PROJECTS\Gait analysis\Dataset\Gait_analysis_cleaned.csv")
#df.head(50)

In [4]:

X = df.drop(columns=['Person_ID'])  
Y = df['Person_ID']



In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report  # ✅ Import classification_report

from sklearn.model_selection import StratifiedKFold

model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = RandomForestClassifier(max_depth=10, min_samples_leaf= 1, min_samples_split=2, n_estimators=300)
model3 = RandomForestClassifier(
    max_depth=5,                # Restrict depth to prevent memorization
    min_samples_split=5,        # Require at least 5 samples to split a node
    min_samples_leaf=3,         # Require at least 3 samples per leaf
    n_estimators=200,           # Reduce the number of trees to 200 (avoid excessive complexity)
    random_state=42
)

# Define Stratified K-Fold (ensures balanced distribution)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Store accuracy scores
scores_m1 = []
scores_m2 = []
scores_m3 = []

# Perform Stratified k-Fold Cross-Validation
for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    # Initialize Models
    model1 = RandomForestClassifier(n_estimators=100, random_state=42)
    model2 = RandomForestClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=300)
    model3 = RandomForestClassifier(max_depth=5, min_samples_split=5, min_samples_leaf=3, n_estimators=200, random_state=42)

    # Train models
    model1.fit(X_train, Y_train)
    model2.fit(X_train, Y_train)
    model3.fit(X_train, Y_train)

    # Predict
    Y_pred_m1 = model1.predict(X_test)
    Y_pred_m2 = model2.predict(X_test)
    Y_pred_m3 = model3.predict(X_test)

    # Store accuracy scores
    scores_m1.append(accuracy_score(Y_test, Y_pred_m1))
    scores_m2.append(accuracy_score(Y_test, Y_pred_m2))
    scores_m3.append(accuracy_score(Y_test, Y_pred_m3))

# Print final results
print(f"Stratified k-Fold CV Accuracy - Model 1 (Baseline): {np.mean(scores_m1):.4f} ± {np.std(scores_m1):.4f}")
print(f"Stratified k-Fold CV Accuracy - Model 2 (Optimized): {np.mean(scores_m2):.4f} ± {np.std(scores_m2):.4f}")
print(f"Stratified k-Fold CV Accuracy - Model 3 (Regularized): {np.mean(scores_m3):.4f} ± {np.std(scores_m3):.4f}")


Stratified k-Fold CV Accuracy - Model 1 (Baseline): 0.9792 ± 0.0295
Stratified k-Fold CV Accuracy - Model 2 (Optimized): 0.9792 ± 0.0295
Stratified k-Fold CV Accuracy - Model 3 (Regularized): 1.0000 ± 0.0000


In [6]:
from sklearn.model_selection import train_test_split

# Ensure the dataset is shuffled properly before splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, stratify=df["Person_ID"], random_state=42)

# Initialize models
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = RandomForestClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=300)
model3 = RandomForestClassifier(max_depth=5, min_samples_split=5, min_samples_leaf=3, n_estimators=200, random_state=42)

# Train models
model1.fit(X_train, Y_train)
model2.fit(X_train, Y_train)
model3.fit(X_train, Y_train)

# Predict
Y_pred_m1 = model1.predict(X_test)
Y_pred_m2 = model2.predict(X_test)
Y_pred_m3 = model3.predict(X_test)

# Print accuracy
print(f"Holdout Test Accuracy - Model 1 (Baseline): {accuracy_score(Y_test, Y_pred_m1):.4f}")
print(f"Holdout Test Accuracy - Model 2 (Optimized): {accuracy_score(Y_test, Y_pred_m2):.4f}")
print(f"Holdout Test Accuracy - Model 3 (Regularized): {accuracy_score(Y_test, Y_pred_m3):.4f}")


Holdout Test Accuracy - Model 1 (Baseline): 1.0000
Holdout Test Accuracy - Model 2 (Optimized): 1.0000
Holdout Test Accuracy - Model 3 (Regularized): 1.0000


COMPARING FEATURE IMPORTANCE ACROSS MODELS

In [17]:
import numpy as np
import pandas as pd

# Get feature importance from all models
feature_importances_m1 = model1.feature_importances_
feature_importances_m2 = model2.feature_importances_
feature_importances_m3 = model3.feature_importances_

# Get feature names
feature_names = X.columns

# Sort features by importance
sorted_indices_m1 = np.argsort(feature_importances_m1)[::-1]
sorted_indices_m2 = np.argsort(feature_importances_m2)[::-1]
sorted_indices_m3 = np.argsort(feature_importances_m3)[::-1]

# Get top 10 features for each model
top_features_m1 = feature_names[sorted_indices_m1[:10]]
top_features_m2 = feature_names[sorted_indices_m2[:10]]
top_features_m3 = feature_names[sorted_indices_m3[:10]]

# Create a DataFrame for comparison
feature_df = pd.DataFrame({
    "Model 1 (Baseline)": top_features_m1,
    "Model 2 (Optimized)": top_features_m2,
    "Model 3 (Regularized)": top_features_m3
})

# Print the feature comparison
print("Top 10 Features Across Models:\n")
print(feature_df)


Top 10 Features Across Models:

  Model 1 (Baseline) Model 2 (Optimized) Model 3 (Regularized)
0                114                 116                   226
1                141                 216                   251
2                178                 251                   220
3                176                 222                   114
4                100                 115                   250
5                 84                 239                    36
6                226                 267                   178
7                250                 304                   163
8                220                 220                   100
9                 35                  82                   318


In [18]:
# Find common features among the top 10 features of all three models
common_features = set(top_features_m1) & set(top_features_m2) & set(top_features_m3)

# Convert to DataFrame for display
common_features_df = pd.DataFrame({"Common Features": list(common_features)})

# Display the common features
print("Common Features in All Models:")
print(common_features_df)

Common Features in All Models:
  Common Features
0             220


In [9]:
import numpy as np
import pandas as pd

# Get feature importance from Model 3
feature_importances_m3 = model3.feature_importances_

# Get feature names
feature_names = X.columns

# Sort features by importance
sorted_indices = np.argsort(feature_importances_m3)[::-1]

# Get the top 10 most important features for Model 3
top_features_m3 = feature_names[sorted_indices[:10]]
top_importances_m3 = feature_importances_m3[sorted_indices[:10]]

# Print the top 10 features
print("Top 10 Most Important Features for Model 3:")
for feature, importance in zip(top_features_m3, top_importances_m3):
    print(f"{feature}: {importance:.4f}")


Top 10 Most Important Features for Model 3:
226: 0.0157
251: 0.0137
220: 0.0115
114: 0.0112
250: 0.0097
36: 0.0095
178: 0.0095
163: 0.0090
100: 0.0086
318: 0.0084


In [10]:
feature_importance_per_person = {}

# Loop through each Person_ID
for person in Y.unique():
    # Convert Person_ID into a binary classification task (1 for this person, 0 for others)
    Y_binary = (Y == person).astype(int)

    # Train Model 3 only for this person
    model_person = RandomForestClassifier(
        max_depth=5, min_samples_split=5, min_samples_leaf=3, n_estimators=200, random_state=42
    )
    model_person.fit(X, Y_binary)

    # Get feature importance
    importance = model_person.feature_importances_
    
    # Store top 5 features for this person
    top_features_person = X.columns[np.argsort(importance)[::-1][:5]]
    feature_importance_per_person[person] = top_features_person

# Convert to DataFrame for better visualization
feature_person_df = pd.DataFrame.from_dict(feature_importance_per_person, orient="index", columns=[f"Feature_{i+1}" for i in range(5)])

# Print the result
print("\nTop Features Used Per Person by Model 3:")
print(feature_person_df)



Top Features Used Per Person by Model 3:
     Feature_1 Feature_2 Feature_3 Feature_4 Feature_5
0.0        249       250       251       252       257
1.0        297       170       162        97        19
2.0         61       181        45       199       129
3.0        226       114       228       222       219
4.0        226       132       249       145         5
5.0        113       145       146       136       310
6.0        107        55       281        82       161
7.0        285       177       287       293       309
8.0        114       120       113       125       232
9.0        178       282       186        22       176
10.0       272       129       300       168       286
11.0       155       161        24       162        99
12.0       205       141       317       226       126
13.0       282       276       307       280       119
14.0       287       177       265       178       275
15.0       315       293       155       249        58


In [11]:
# Flatten all selected features
all_selected_features = feature_person_df.values.flatten()

# Count occurrences of each feature
feature_counts = pd.Series(all_selected_features).value_counts()

# Print the most commonly used features
print("\nMost Commonly Used Features Across Individuals:")
print(feature_counts.head(10))



Most Commonly Used Features Across Individuals:
249    3
226    3
129    2
162    2
161    2
113    2
145    2
114    2
178    2
293    2
Name: count, dtype: int64
